In [2]:
import numpy as np
import pandas as pd
import pickle

In [3]:
# Next day return
stock_price = pd.read_excel('796 price-data.xlsx')
stock_price = stock_price[stock_price.Date>='2007-01-01']
stock_price = stock_price.set_index('Date')
stock_price = stock_price.fillna(method='bfill') # bfill N/A
return_next_date = stock_price.pct_change().shift(-1)
return_next_date.dropna(inplace=True)

In [4]:
# Macro
macro = pd.read_excel('796 macro-data.xlsx')
macro = macro[macro.Date>='2006-01-01']
macro = macro.set_index('Date')
macro = macro.fillna(method='bfill')

In [5]:
macro_lag = 12
macro_monthly = macro.resample('M', convention='end').ffill()
macro_data = np.zeros((return_next_date.shape[0],macro_lag+1,macro.shape[1]))
for j in range(0,macro_lag+1):
    tmp = pd.DataFrame(index=macro.index)
    tmp = pd.DataFrame.merge(tmp,macro_monthly.shift(j),how='left',on='Date')
    tmp = tmp.fillna(method='bfill').loc[return_next_date.index,:]
    tmp = tmp.fillna(macro_monthly.shift(11).loc['2020-04-30'])
    macro_data[:,j,:] = tmp

In [8]:
wb = pd.ExcelFile('796 micro-data.xlsx')
sheets = wb.sheet_names

In [9]:
df = wb.parse(sheets[3])
df = df.set_index('Date').loc['2007-01-04':'2020-04-17',:]
micro_data = np.zeros((df.shape[0],df.shape[1],len(sheets)))
i = 0
for s in sheets:
    df = wb.parse(s)
    df = df.set_index('Date').loc['2007-01-04':'2020-04-17',:]
    df = df.fillna(method='bfill')
    micro_data[:,:,i] = df.values
    i = i + 1

In [10]:
return_data = return_next_date.values

In [11]:
file = open('a1', 'wb')
pickle.dump([macro_data,micro_data,return_next_date], file)
file.close()